In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

## 第12章 pandas高级应用
前面的章节关注于不同类型的数据规整流程和NumPy、pandas与其它库的特<br/>点。随着时间的发展，pandas发展出了更多适合高级用户的功能。本章<br/>就要深入学习pandas的高级功能。

### 12.1 分类数据
这一节介绍的是pandas的分类类型。我会向你展示通过使用它，提高性能和内<br/>存的使用率。我还会介绍一些在统计和机器学习中使用分类数据的工具。

#### 背景和目的
表中的一列通常会有重复的包含不同值的小集合的情况。我们已经学过了<br/>unique和value_counts，它们可以从数组提取出不同的值，并分别<br/>计算频率

In [1]:
import numpy as np
import pandas as pd

values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)

values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [2]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [3]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [4]:
#许多数据系统（数据仓库、统计计算或其它应用）都发展出了特定的表征重
#复值的方法，以进行高效的存储和计算。在数据仓库中，最好的方法是使用
#所谓的包含不同值的维表(Dimension Table)，将主要的参数存储为引用
#维表整数键
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(['apple', 'orange'])

values
#用0代表apple，1代表orange

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [5]:
dim

0     apple
1    orange
dtype: object

In [6]:
#可以使用take方法存储原始的字符串Series
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种用整数表示的方法称为分类或字典编码表示法。不同值得数组称为分类、<br/>字典或数据级。本书中，我们使用分类的说法。表示分类的整数值称为<br/>分类编码或简单地称为编码。
<br/>分类表示可以在进行分析时大大的提高性能。你也可以在保持编码不变<br/>的情况下，对分类进行转换。一些相对简单的转变例子包括：
<br/>重命名分类。
<br/>加入一个新的分类，不改变已经存在的分类的顺序或位置。

#### pandas的分类类型
pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据。看一个<br/>之前的Series例子

In [9]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

N = len(fruits)

df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                   columns=['basket_id', 'fruit', 'count', 'weight'])

df

,basket_id,fruit,count,weight
0,0,apple,11,3.370380
1,1,orange,8,3.891471
2,2,apple,5,1.333134
3,3,apple,9,0.602186
4,4,apple,13,0.873914
5,5,orange,11,3.892768
6,6,apple,5,2.175338
7,7,apple,14,3.007098


In [10]:
#这里，df['fruit']是一个Python字符串对象的数组。我们可以通过调用它，
#将它转变为分类
fruit_cat = df['fruit'].astype('category')

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [11]:
#fruit_cat的值不是NumPy数组，而是一个pandas.Categorical实例
c = fruit_cat.values

type(c)

pandas.core.categorical.Categorical

In [12]:
#分类对象有categories和codes属性
c.categories

Index(['apple', 'orange'], dtype='object')

In [13]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [15]:
#你可将DateFrame的列通过分配转换结果，转换为分类
df['fruit'] = df['fruit'].astype('category')

df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [17]:
#你还可以从其它Python序列直接创建pandas.Categorical
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

my_categories


[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [19]:
#如果你已经从其它源获得了分类编码，你还可以使用from_codes构造器
categories = ['foo', 'bar', 'baz']

codes = [0, 1, 2, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)

my_cats_2
#foo为0，bar为1，baz为2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [22]:
#与显示指定不同，分类变换不认定指定的分类顺序。因此取决于输入数据的顺
#序，categories数组的顺序会不同。当使用from_codes或其它的构造器时，
#你可以指定分类一个有意义的顺序：

ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)

ordered_cat
# print(ordered_cat.codes)

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [25]:
#输出[foo < bar < baz]指明‘foo’位于‘bar’的前面，以此类推。
#无序的分类实例可以通过as_ordered排序
my_cats_2.as_ordered()

#最后要注意，分类数据不需要字符串，尽管我仅仅展示了字符串的例子。
#分类数组可以包括任意不可变类型。

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

#### 用分类进行计算
与非编码版本（比如字符串数组）相比，使用pandas的Categorical有些类<br/>似。某些pandas组件，比如groupby函数，更适合进行分类。还有一<br/>些函数可以使用有序标志位。
<br/>来看一些随机的数值数据，使用pandas.qcut面元函数。它会返回<br/>pandas.Categorical，我们之前使用过pandas.cut，但没解释分<br/>类是如何工作的

In [26]:
np.random.seed(12345)
draws = np.random.randn(1000)

draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [27]:
#计算这个数据的分位面元，提取一些统计信息
bins = pd.qcut(draws, 4)

bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.95, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.95, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [28]:
#虽然有用，确切的样本分位数与分位的名称相比，不利于生成汇总。
#我们可以使用labels参数qcut，实现目的
bins = pd.qcut(draws, 4,labels=['Q1', 'Q2', 'Q3', 'Q4'])

bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [29]:
bins.codes[:10]
#Q1为0，Q2为1，Q3为2，Q4为3

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [39]:
#加上标签的面元分类不包含数据面元边界的信息，因此可以使用groupby提取
#一些汇总信息
bins = pd.Series(bins, name='quartile')

results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())

results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [40]:
#分位数列保存了原始的面元分类信息，包括排序
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### 用分类提高性能
如果你是在一个特定数据集上做大量分析，将其转换为分类可以极大地提高效<br/>率。DataFrame列的分类使用的内存通常少的多。来看一些包含一千万<br/>元素的Series，和一些不同的分类：

In [46]:
N = 10000000

draws = pd.Series(np.random.randn(N))

labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [47]:
#现在，将标签转换为分类
categories = labels.astype('category')

In [48]:
#这时，可以看到标签使用的内存远比分类多
labels.memory_usage()

80000080

In [50]:
categories.memory_usage()

10000272

In [52]:
#转换为分类不是没有代价的，但这是一次性的代价
%time _ = labels.astype('category')

CPU times: user 344 ms, sys: 62.4 ms, total: 407 ms
Wall time: 414 ms


#### 分类方法
包含分类数据的Series有一些特殊的方法，类似于Series.str字符串方法。<br/>它还提供了方便的分类和编码的使用方法。看下面的Series

In [53]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

cat_s = s.astype('category')

cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [54]:
#特别的cat属性提供了分类方法的入口
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [55]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [56]:
#假设我们知道这个数据的实际分类集，超出了数据中的四个值。
#我们可以使用set_categories方法改变它们
actual_categories = ['a', 'b', 'c', 'd', 'e']

cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [57]:
#虽然数据看起来没变，新的分类将反映在它们的操作中。例如，如果有的话，
#value_counts表示分类
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [58]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [59]:
#在大数据集中，分类经常作为节省内存和高性能的便捷工具。过滤完
#大DataFrame或Series之后，许多分类可能不会出现在数据中。我们可以
#使用remove_unused_categories方法删除没看到的分类
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [60]:
cat_s3.cat.remove_unused_categories()
#删除掉cat_s3中没有的分类

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

#### 为建模创建虚拟变量
当你使用统计或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为<br/>one-hot编码。这包括创建一个不同类别的列的DataFrame；这些列包<br/>含给定分类的1s，其它为0

In [61]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [62]:
#前面的第7章提到过，pandas.get_dummies函数可以转换这个分类数据为
#包含虚拟变量的DataFrame
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


### 12.2 GroupBy高级应用
尽管我们在第10章已经深度学习了Series和DataFrame的Groupby方法，还<br/>有一些方法也是很有用的

#### 分组转换和“解封”GroupBy
在第10章，我们在分组操作中学习了apply方法，进行转换。还有另一个<br/>transform方法，它与apply很像，但是对使用的函数有一定限制：
<br/>它可以产生向分组形状广播标量值
<br/>它可以产生一个和输入组形状相同的对象
<br/>它不能修改输入
<br/>来看一个简单的例子

In [63]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [64]:
#按键进行分组
g = df.groupby('key').value

g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [67]:
#假设我们想产生一个和df['value']形状相同的Series，但值替换为按键分组的平均
#值。我们可以传递函数lambda x: x.mean()进行转换
g.transform(lambda x: x.mean())
##为什么用g.transfrom能形成df['value']形状相同的Series

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [66]:
#对于内置的聚合函数，我们可以传递一个字符串假名作为GroupBy的agg方法
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [68]:
#与apply类似，transform的函数会返回Series，但是结果必须与输入大小相同。
#举个例子，我们可以用lambda函数将每个分组乘以2
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [69]:
#再举一个复杂的例子，我们可以计算每个分组的降序排名
g.transform(lambda x: x.rank(ascending=False))
#这个排名有点看不太懂

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [70]:
#看一个由简单聚合构造的的分组转换函数
def normalize(x):
    return (x - x.mean()) / x.std()

In [71]:
#我们用transform或apply可以获得等价的结果
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [72]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [73]:
#内置的聚合函数，比如mean或sum，通常比apply函数快，也比transform
#快。这允许我们进行一个所谓的解封（unwrapped）分组操作
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [74]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized
#解封分组操作可能包括多个分组聚合，但是矢量化操作还是会带来收益

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

#### 分组的时间重采样
对于时间序列数据，resample方法从语义上是一个基于内在时间的分组操<br/>作。下面是一个示例表

In [75]:
N = 15

times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [78]:
#这里，我们可以用time作为索引，然后重采样
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [80]:
#假设DataFrame包含多个时间序列，用一个额外的分组键的列进行标记
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})

print(df2)


   key                time  value
0    a 2017-05-20 00:00:00    0.0
1    b 2017-05-20 00:00:00    1.0
2    c 2017-05-20 00:00:00    2.0
3    a 2017-05-20 00:01:00    3.0
4    b 2017-05-20 00:01:00    4.0
5    c 2017-05-20 00:01:00    5.0
6    a 2017-05-20 00:02:00    6.0
7    b 2017-05-20 00:02:00    7.0
8    c 2017-05-20 00:02:00    8.0
9    a 2017-05-20 00:03:00    9.0
10   b 2017-05-20 00:03:00   10.0
11   c 2017-05-20 00:03:00   11.0
12   a 2017-05-20 00:04:00   12.0
13   b 2017-05-20 00:04:00   13.0
14   c 2017-05-20 00:04:00   14.0
15   a 2017-05-20 00:05:00   15.0
16   b 2017-05-20 00:05:00   16.0
17   c 2017-05-20 00:05:00   17.0
18   a 2017-05-20 00:06:00   18.0
19   b 2017-05-20 00:06:00   19.0
20   c 2017-05-20 00:06:00   20.0
21   a 2017-05-20 00:07:00   21.0
22   b 2017-05-20 00:07:00   22.0
23   c 2017-05-20 00:07:00   23.0
24   a 2017-05-20 00:08:00   24.0
25   b 2017-05-20 00:08:00   25.0
26   c 2017-05-20 00:08:00   26.0
27   a 2017-05-20 00:09:00   27.0
28   b 2017-05

In [87]:
#要对每个key值进行相同的重采样，我们引入pandas.TimeGrouper对象
time_key = pd.TimeGrouper('5min')

In [88]:
#我们然后设定时间索引，用key和time_key分组，然后聚合
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())

resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [89]:
resampled.reset_index()
#使用TimeGrouper的限制是时间必须是Series或DataFrame的索引

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


### 12.3 链式编程技术
当对数据集进行一系列变换时，你可能发现创建的多个临时变量其实并没有在<br/>分析中用到。看下面的例子

In [91]:
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()

虽然这里没有使用真实的数据，这个例子却指出了一些新方法。首先，<br/>DataFrame.assign方法是一个df[k] = v形式的函数式的列分配方<br/>法。它不是就地修改对象，而是返回新的修改过的DataFrame。因此，<br/>下面的语句是等价的

In [ ]:
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)

In [ ]:
#就地分配可能会比assign快，但是assign可以方便地进行链式编程
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())

我使用外括号，这样便于添加换行符。
<br/>使用链式编程时要注意，你可能会需要涉及临时对象。在前面的例子<br/>中，我们不能使用load_data的结果，直到它被赋值给临时变量df。为<br/>了这么做，assign和许多其它pandas函数可以接收类似函数的参数，<br/>即可调用对象（callable）。为了展示可调用对象，看一个前面例子<br/>的片段

In [ ]:
df = load_data()
df2 = df[df['col2'] < 0]

In [ ]:
#它可以重写为
df = (load_data()
      [lambda x: x['col2'] < 0])

这里，load_data的结果没有赋值给某个变量，因此传递到[ ]的函数在这一<br/>步被绑定到了对象。
<br/>我们可以把整个过程写为一个单链表达式

In [ ]:
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
#是否将代码写成这种形式只是习惯而已，将它分开成若干步可以提高可读性

#### 管道方法
你可以用Python内置的pandas函数和方法，用带有可调用对象的链式编程做<br/>许多工作。但是，有时你需要使用自己的函数，或是第三方库的函数。<br/>这时就要用到管道方法

In [ ]:
#看下面的函数调用
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)

In [ ]:
#当使用接收、返回Series或DataFrame对象的函数式，你可以调用pipe将
#其重写
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))

In [ ]:
#f(df)和df.pipe(f)是等价的，但是pipe使得链式声明更容易。
#pipe的另一个有用的地方是提炼操作为可复用的函数。看一个从列减去分组
#方法的例子
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')

In [ ]:
#假设你想转换多列，并修改分组的键。另外，你想用链式编程做这个转换。
#下面就是一个方法
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result

In [ ]:
#然后可以写为
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))

In [96]:
N = len(fruits)

df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                   columns=['basket_id', 'fruit', 'count', 'weight'])

df

,basket_id,fruit,count,weight
0,0,apple,12,2.061554
1,1,orange,13,2.354739
2,2,apple,12,2.201177
3,3,apple,8,3.748274
4,4,apple,6,1.087926
5,5,orange,6,0.954187
6,6,apple,11,1.806502
7,7,apple,11,3.066555


In [99]:
df2 = df.assign(count=0)
df2

,basket_id,fruit,count,weight
0,0,apple,0,2.061554
1,1,orange,0,2.354739
2,2,apple,0,2.201177
3,3,apple,0,3.748274
4,4,apple,0,1.087926
5,5,orange,0,0.954187
6,6,apple,0,1.806502
7,7,apple,0,3.066555


### 12.4 总结